In [1]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\janlu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install emd

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\janlu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install pingouin

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\janlu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\janlu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install sails

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\janlu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import emd.sift as sift
import emd.spectra as spectra
import numpy as np
import pingouin as pg
import sails
import scipy.io as sio
import h5py
import time
import timeit
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy.signal import convolve2d
from scipy.stats import zscore, binned_statistic
from scipy.ndimage import center_of_mass
import matplotlib.pyplot as plt
import os
import glob
import concurrent.futures
from skimage.feature import peak_local_max


In [2]:
def get_rem_states(states_data, sample_rate):
    states_data = np.squeeze(states_data)
    rem_state_indices = np.where(states_data == 5)[0]
    rem_state_changes = np.diff(rem_state_indices)
    split_indices = np.where(rem_state_changes != 1)[0] + 1
    split_indices = np.concatenate(([0], split_indices, [len(rem_state_indices)]))
    consecutive_rem_states = np.empty((len(split_indices) - 1, 2))
    for i, (start, end) in enumerate(zip(split_indices, split_indices[1:])):
        start = rem_state_indices[start] * int(sample_rate)
        end = rem_state_indices[end - 1] * int(sample_rate)
        consecutive_rem_states[i] = np.array([start, end])
    consecutive_rem_states = np.array(consecutive_rem_states)
    null_states_mask = np.squeeze(np.diff(consecutive_rem_states) > 0)
    consecutive_rem_states = consecutive_rem_states[null_states_mask]
    return consecutive_rem_states


def morlet_wt(x, sample_rate, frequencies=np.arange(1, 200, 1), n=5, mode='complex'):
    wavelet_transform = sails.wavelet.morlet(x, freqs=frequencies, sample_rate=sample_rate, ncycles=n,
                                             ret_mode=mode, normalise=None)
    return wavelet_transform


def tg_split(mask_freq, theta_range=(5, 12)):
    lower = np.min(theta_range)
    upper = np.max(theta_range)
    mask_index = np.logical_and(mask_freq >= lower, mask_freq < upper)
    sub_mask_index = mask_freq < lower
    supra_mask_index = mask_freq > upper
    sub = sub_mask_index
    theta = mask_index
    supra = supra_mask_index

    return sub, theta, supra


def zero_cross(x):
    decay = np.logical_and((x > 0)[1:], ~(x > 0)[:-1]).nonzero()[0]
    rise = np.logical_and((x <= 0)[1:], ~(x <= 0)[:-1]).nonzero()[0]
    zero_xs = np.sort(np.append(rise, decay))
    return zero_xs


def extrema(x):
    zero_xs = zero_cross(x)
    peaks = np.empty((0,)).astype(int)
    troughs = np.empty((0,)).astype(int)
    for t1, t2 in zip(zero_xs, zero_xs[1:]):
        extrema0 = np.argmax(np.abs(x[t1:t2])).astype(int) + t1
        if bool(x[extrema0] > 0):
            peaks = np.append(peaks, extrema0)
        else:
            troughs = np.append(troughs, extrema0)
    return zero_xs, troughs, peaks


def get_cycles_data(x, rem_states, sample_rate, theta_range=(5, 12)):
    consecutive_rem_states = get_rem_states(rem_states, sample_rate)
    rem_imf = []
    rem_mask_freq = []
    instantaneous_phase = []
    instantaneous_freq = []
    instantaneous_amp = []
    sub_theta_sig = np.empty((0,), dtype=np.float32)  # Specify dtype here
    theta_peak_sig = np.empty((0,), dtype=np.float32)  # Specify dtype here
    cycles = np.empty((0, 5), dtype=np.int32)  # Specify dtype here
    rem_dict = {}
    sub_dict = rem_dict

    for i, rem in enumerate(consecutive_rem_states, start=1):
        sub_dict.setdefault(f'REM {i}', {})
        start = int(rem[0])
        end = int(rem[1])
        signal = x[start:end]
        imf, mask_freq = sift.iterated_mask_sift(signal,
                                                 mask_0='zc',
                                                 sample_rate=sample_rate,
                                                 ret_mask_freq=True)
        IP, IF, IA = spectra.frequency_transform(imf, sample_rate, 'nht')
        sub_theta, theta, _ = tg_split(mask_freq, theta_range)

        rem_imf.append(imf)
        rem_mask_freq.append(mask_freq)
        instantaneous_phase.append(IP)
        instantaneous_freq.append(IF)
        instantaneous_amp.append(IA)

        theta_sig = np.sum(imf.T[theta], axis=0)
        sub_theta_sig = np.append(sub_theta_sig, np.sum(imf.T[sub_theta], axis=0))

        zero_x, trough, peak = extrema(np.sum(imf.T[theta], axis=0))

        zero_x = np.vstack((zero_x[:-2:2], zero_x[1:-1:2], zero_x[2::2])).T

        size_adjust = np.min([trough.shape[0], zero_x.shape[0], peak.shape[0]])
        zero_x = zero_x[:size_adjust]
        cycle = np.empty((size_adjust, 5), dtype=np.int32)  # Specify dtype here
        cycle[:, [0, 2, 4]] = zero_x
        if trough[0] < peak[0]:
            cycle[:, 1] = trough[:zero_x.shape[0]]
            cycle[:, 3] = peak[:zero_x.shape[0]]
        else:
            cycle[:, 3] = trough[:zero_x.shape[0]]
            cycle[:, 1] = peak[:zero_x.shape[0]]

        broken_cycle = cycle[~np.all(np.diff(cycle, axis=1) > 0, axis=1)]
        broken_cycle_mask = np.diff(broken_cycle, axis=1) > 0

        adjust_condition = np.all(np.all(broken_cycle_mask[1:] == [True, False, False, True],
                                         axis=0) == True)
        adjust_loc = np.where(np.all(np.diff(cycle, axis=1) > 0, axis=1) == False)[0][1:-1]

        fixed_cycle = broken_cycle[1:-1]
        if adjust_condition:
            fixed_cycle[:, 1] = cycle[adjust_loc - 1, 1]
            fixed_cycle[:, 3] = cycle[adjust_loc + 1, 3]
        else:
            fixed_cycle[:, 3] = cycle[adjust_loc - 1, 3]
            fixed_cycle[:, 1] = cycle[adjust_loc + 1, 1]

        cycle = cycle[np.all(np.diff(cycle, axis=1) > 0, axis=1)]
        cycle = np.vstack((cycle, fixed_cycle))
        if trough[0] < peak[0]:
            cycle = np.hstack((cycle[:-1, 1:-1], cycle[1:, :2]))
        else:
            cycle = np.hstack((cycle[:-1, 3].reshape((-1, 1)), cycle[1:, :-1]))

        theta_peak_sig = np.append(theta_peak_sig, theta_sig[cycle[:, 2]])
        cycles = np.vstack((cycles, cycle + start))

    min_peak_amp = 2 * sub_theta_sig.std()
    peak_mask = theta_peak_sig > min_peak_amp
    upper_diff = np.floor(1000 / np.min(theta_range))
    lower_diff = np.floor(1000 / np.max(theta_range))
    diff_mask = np.logical_and(np.diff(cycles[:, [0, -1]], axis=1) * (1000 / sample_rate) > lower_diff,
                               np.diff(cycles[:, [0, -1]], axis=1) * (1000 / sample_rate) <= upper_diff)

    extrema_mask = np.logical_and(np.squeeze(diff_mask), peak_mask)

    cycles = cycles[extrema_mask]

    for j, rem in enumerate(rem_dict.values()):
        rem['start-end'] = consecutive_rem_states[j].astype(np.int32)
        rem['IMFs'] = rem_imf[j]
        rem['IMF_Frequencies'] = rem_mask_freq[j]
        rem['Instantaneous Phases'] = instantaneous_phase[j]
        rem['Instantaneous Frequencies'] = instantaneous_freq[j]
        rem['Instantaneous Amplitudes'] = instantaneous_amp[j]
        cycles_mask = (cycles > consecutive_rem_states[j, 0]) & (cycles < consecutive_rem_states[j, 1])
        cycles_mask = np.all(cycles_mask == True, axis=1)
        rem_cycles = cycles[cycles_mask]
        rem['Cycles'] = rem_cycles.astype(np.int32)
    return rem_dict




def bin_tf_to_fpp(x, power, bin_count):
    if x.ndim == 1:  # Handle the case when x is of size (2)
        bin_ranges = np.arange(x[0], x[1], 1)
        fpp = binned_statistic(bin_ranges, power[:, x[0]:x[1]], 'mean', bins=bin_count)[0]
        fpp = np.expand_dims(fpp, axis=0)  # Add an extra dimension to match the desired output shape
    elif x.ndim == 2:  # Handle the case when x is of size (n, 2)
        fpp = []
        for i in range(x.shape[0]):
            bin_ranges = np.arange(x[i, 0], x[i, 1], 1)
            fpp_row = binned_statistic(bin_ranges, power[:, x[i, 0]:x[i, 1]], 'mean', bins=bin_count)[0]
            fpp.append(fpp_row)
        fpp = np.array(fpp)
    else:
        raise ValueError("Invalid size for x")

    return fpp


def calculate_cog(frequencies, angles, amplitudes, ratio):
    angles = np.deg2rad(angles)
    cog = np.empty((0, 2))
    if amplitudes.ndim == 2:
        numerator = np.sum(frequencies * np.sum(amplitudes, axis=1))
        denominator = np.sum(amplitudes)
        cog_f = numerator / denominator
        floor = np.floor(cog_f).astype(int) - frequencies[0]
        ceil = np.ceil(cog_f).astype(int) - frequencies[0]
        new_fpp = np.where(amplitudes >= np.max(amplitudes[[floor, ceil], :]) * ratio, amplitudes, 0)
        cog_ph = np.rad2deg(pg.circ_mean(angles, w=np.sum(new_fpp, axis=0)))
        cog = np.array([cog_f, cog_ph])
    elif amplitudes.ndim == 3:
        indices_to_subset = np.empty((amplitudes.shape[0], 2)).astype(int)
        cog = np.empty((amplitudes.shape[0], 2))
        numerator = np.sum(frequencies * np.sum(amplitudes, axis=2), axis=1)
        denominator = np.sum(amplitudes, axis=(1, 2))
        cog_f = (numerator / denominator)
        vectorized_floor = np.vectorize(np.floor)
        vectorized_ceil = np.vectorize(np.ceil)
        indices_to_subset[:, 0] = vectorized_floor(cog_f) - frequencies[0]
        indices_to_subset[:, 1] = vectorized_ceil(cog_f) - frequencies[0]
        max_amps = np.max(amplitudes[np.arange(amplitudes.shape[0])[:, np.newaxis], indices_to_subset, :], axis=(1, 2))
        print(max_amps.shape)
        for i, max_amp in enumerate(max_amps):
            new_fpp = np.where(amplitudes[i] >= max_amp * ratio, amplitudes[i], 0)
            cog[i, 1] = np.rad2deg(pg.circ_mean(angles, w=np.sum(new_fpp, axis=0)))
        cog[:, 0] = cog_f
    return cog


def boxcar_smooth(x, boxcar_window):
    if x.ndim == 1:
        if boxcar_window % 2 == 0:
            boxcar_window += 1
        window = np.ones((1, boxcar_window)) / boxcar_window
        x_spectrum = np.convolve(x, window, mode='same')
    else:
        bool_window = np.where(~boxcar_window % 2 == 0, boxcar_window, boxcar_window + 1)
        window_t = np.ones((1, bool_window[0])) / bool_window[0]
        window_f = np.ones((1, bool_window[1])) / bool_window[1]
        x_spectrum_t = convolve2d(x, window_t, mode='same')
        x_spectrum = convolve2d(x_spectrum_t, window_f.T, mode='same')

    return x_spectrum


# def peak_cog(frequencies, angles, amplitudes, ratio):
#     def nearest_peaks(frequency, angle, amplitude, ratio):
#         peak_indices = peak_local_max(amplitude, min_distance=1, threshold_abs=0)
#         cog_f = calculate_cog(frequency, angle, amplitude, ratio)

#         if peak_indices.shape[0] == 0:
#             cog_peak = cog_f
#         else:
#             cog_fx = np.array([cog_f[0], cog_f[0] * np.cos(np.deg2rad(cog_f[1] - angle[0])),
#                                cog_f[0] * np.sin(np.deg2rad(cog_f[1] - angle[0]))])
#             peak_loc = peak_loc = np.empty((peak_indices.shape[0], 4))
#             peak_loc[:, [0, 1]] = np.array([frequency[peak_indices.T[0]], angle[peak_indices.T[1]]]).T
#             peak_loc[:, 2] = peak_loc[:, 0] * np.cos(np.deg2rad(peak_loc[:, 1] - angle[0]))
#             peak_loc[:, 3] = peak_loc[:, 0] * np.sin(np.deg2rad(peak_loc[:, 1] - angle[0]))
#             peak_loc = peak_loc[:, [0, 2, 3]]
#             distances = np.abs(peak_loc - cog_fx)

#             cog_pos = peak_indices[np.argmin(np.linalg.norm(distances, axis=1))]

#             cog_peak = np.array([frequency[cog_pos[0]], angle[cog_pos[1]]])

#         return cog_peak

#     if amplitudes.ndim == 2:
#         cog = nearest_peaks(frequencies, angles, amplitudes, ratio)
#     elif amplitudes.ndim == 3:
#         cog = np.empty((amplitudes.shape[0], 2))
#         for i, fpp in enumerate(amplitudes):
#             cog[i] = nearest_peaks(frequencies, angles, fpp, ratio)
#     return cog


# def max_peaks(amplitudes):
#     new_fpp = np.zeros(amplitudes.shape)
#     if amplitudes.ndim == 2:
#         peak_indices = peak_local_max(amplitudes, min_distance=1, threshold_abs=0)
#         if peak_indices.shape[0] == 0:
#             new_fpp = np.where(amplitudes > 0, amplitudes, 0)
#         else:
#             new_fpp[peak_indices.T[0], peak_indices.T[1]] = amplitudes[peak_indices.T[0], peak_indices.T[1]]
#     elif amplitudes.ndim == 3:
#         for i, fpp in enumerate(amplitudes):
#             peak_indices = peak_local_max(fpp, min_distance=1, threshold_abs=0)
#             if peak_indices.shape[0] == 0:
#                 new_fpp[i] = np.where(fpp > 0, fpp, 0)
#             else:
#                 new_fpp[i, peak_indices.T[0], peak_indices.T[1]] = fpp[peak_indices.T[0], peak_indices.T[1]]
#     return new_fpp


# def boundary_peaks(amplitudes):
#     adjusted_fpp = np.zeros(amplitudes.shape)
#     if amplitudes.ndim == 2:
#         peak_indices = peak_local_max(amplitudes, min_distance=1, threshold_abs=0)
#         if peak_indices.shape[0] == 0:
#             adjusted_fpp = np.where(amplitudes > 0, amplitudes, 0)
#         else:
#             new_fpp = amplitudes[peak_indices.T[0], peak_indices.T[1]]
#             maximum = np.max(new_fpp)
#             minimum = np.min(new_fpp)
#             adjusted_fpp = np.where((amplitudes <= maximum) & (amplitudes >= 0.95*minimum), amplitudes, 0)
#     elif amplitudes.ndim == 3:
#         for i, fpp in enumerate(amplitudes):
#             peak_indices = peak_local_max(fpp, min_distance=1, threshold_abs=0)
#             print(peak_indices.shape)
#             if peak_indices.shape[0] == 0:
#                 adjusted_fpp[i] = np.where(fpp > 0, fpp, 0)
#             else:
#                 maximum = np.max(fpp[peak_indices.T[0], peak_indices.T[1]])
#                 minimum = np.min(fpp[peak_indices.T[0], peak_indices.T[1]])
#                 adjusted_fpp[i] = np.where((fpp <= maximum) & (fpp >= 0.95*minimum), fpp, 0)
#     return adjusted_fpp


import numpy as np

def rem_fpp_gen(rem_dict, x, sample_rate, frequencies, angles, ratio, boxcar_window=None, norm='', fpp_method='',
                cog_method=''):
    x = np.squeeze(x)
    cycles_dict = rem_dict
    rem_dict = {}
    sub_dict = rem_dict
    for key, value in cycles_dict.items():
        print(key)
        if 'Cycles' in value.keys():
            sub_dict.setdefault(key, {})
            t = value['start-end'].astype(np.int32)
            print(t, t[0], t[1])
            sig = x[t[0]:t[1]]
            print(sig.shape)
            power = morlet_wt(sig, sample_rate, frequencies, mode='power').astype(np.float32)
            cycles = (value['Cycles'][:, [0, -1]] - t[0]).astype(np.int32)
            # if boxcar_window is not None:
            #     power = boxcar_smooth(power, boxcar_window)
            # if norm == 'simple_x':
            #     power = power / np.sum(power, axis=0)
            # elif norm == 'simple_y':
            #     power = power / np.sum(power, axis=1)[:, np.newaxis]
            # elif norm == 'zscore_y':
            #     power = zscore(power, axis=0)
            # elif norm == 'zscore_x':
            #     power = zscore(power, axis=1)
            fpp_plots = bin_tf_to_fpp(cycles, power, 19).astype(np.float32)
            sub_dict[key]['FPP_cycles'] = fpp_plots
            # if fpp_method == 'max_peaks':
            #     fpp_plots = max_peaks(fpp_plots)
            #     print(fpp_plots.shape)
            # elif fpp_method == 'boundary_peaks':
            #     fpp_plots = boundary_peaks(fpp_plots)
            # if cog_method == 'nearest':
            #     cog = peak_cog(frequencies, angles, fpp_plots, ratio).astype(np.float32)
            # else:
            #     cog = calculate_cog(frequencies, angles, fpp_plots, ratio).astype(np.float32)
            # sub_dict[key]['CoG'] = cog
        else:
            continue
    return rem_dict


In [5]:
class DataProcessor:
    def __init__(self, data_dir, output_dir):
        # Initialize the data and output directories.
        self.data_dir = data_dir  # Set the input data directory.
        self.output_dir = output_dir  # Set the output data directory.

    def load_data(self, subfolder):
        # Generate file paths for LFP and states data using glob.
        # Find the file that matches the pattern '*HPC*' in the specified subfolder.
        lfp_file = glob.glob(os.path.join(self.data_dir, subfolder, '*HPC*'))[0]
        # Find the file that matches the pattern '*states*' in the specified subfolder.
        states_file = glob.glob(os.path.join(self.data_dir, subfolder, '*states*'))[0]

        # Load LFP and states data using scipy's loadmat function.
        lfp_data = sio.loadmat(lfp_file)['HPC']  # Load LFP data from the MATLAB file.
        states_data = sio.loadmat(states_file)['states']  # Load states data.
        print(states_data.shape)
        return lfp_data, states_data 
       

    def process_data(self, lfp_data, states_data):
        # Add your data processing pipeline here.
        # For example, you can apply signal processing or any other data transformations.
        # For now, we're just passing through the data as-is.
        rem_dict = get_cycles_data(lfp_data, states_data, 2500, (5, 12))
    
        # MW
        rem_periods = get_rem_states(states_data, 2500).astype(int)
    
        rem_signals=[]
        for rem in rem_periods:
            signal = lfp_data[rem[0]:rem[1]]
            rem_signals.append(np.squeeze(signal))
    
        frequencies = np.arange (20, 140, 1)
        angles = np.linspace(-180, 180, 19)
        rem_tf_power_dict = {}  # Initialize a dictionary to store power for all REM periods

        for i, sig in enumerate(rem_signals):
            power = morlet_wt(sig, 2500, frequencies, mode='power')
    
            # Calculate summed power values for the current REM period
            summed_power_values = np.sum(power, axis=1)
            
            # Convert power and summed_power_values to suitable data types (e.g., float32)
            power = np.array(power, dtype=np.float32)
            summed_power_values = np.array(summed_power_values, dtype=np.float32)
    
            # Store 'power' and 'summed_power_values' for the current REM period in a dictionary
            rem_tf_power_dict[f'REM_period_{i + 1}'] = {
                'tf_power': power,
                'summed_power': summed_power_values
            }
        for key, value in rem_dict.items():
            if isinstance(value, (int, float)):
                rem_dict[key] = np.float32(value)  # Convert numerical values to np.float32
                
        return rem_dict, rem_tf_power_dict

            # Now, rem_tf_power_dict contains both time-frequency power and summed power values for each REM period

          



    def save_data(self, subfolder, rem_dict, rem_tf_power_dict):
        # Create an output subfolder if it doesn't exist.
        output_subfolder = os.path.join(self.output_dir, subfolder)
        os.makedirs(output_subfolder, exist_ok=True)
        
        # Define the output file paths based on the subfolder name.
        rem_dict_filename = f"{subfolder}_REM_dict.h5"
        rem_dict_file = os.path.join(output_subfolder, rem_dict_filename)
        
        rem_tf_power_dict_filename = f"{subfolder}_REM_tf_power_dict.h5"
        rem_tf_power_dict_file = os.path.join(output_subfolder, rem_tf_power_dict_filename)
        
        # Validate data types in rem_dict
        for key, value in rem_dict.items():
            print(f"Key: {key}, Value Type: {type(value)}")  # Add this line for debugging
            if not isinstance(value, (int, float, np.ndarray)):
                raise ValueError(f"Invalid data type in rem_dict for key '{key}': {type(value)}")

        # Validate data types in rem_tf_power_dict
        for key, value in rem_tf_power_dict.items():
            if not isinstance(value['tf_power'], np.ndarray) or not isinstance(value['summed_power'], np.ndarray):
                raise ValueError(f"Invalid data type in rem_tf_power_dict for key '{key}'")

        # Save the rem_dict dictionary as an HDF5 file.
        with h5py.File(rem_dict_file, 'w') as hdf_file:
            # Create a group to store the rem_dict data
            rem_dict_group = hdf_file.create_group('REM_dict_group')

            # Save each key-value pair from rem_dict into the HDF5 group
            for key, value in rem_dict.items():
                # Use key as the dataset name and store the value
                rem_dict_group[key] = value

        # Save the rem_tf_power_dict dictionary as an HDF5 file.
        with h5py.File(rem_tf_power_dict_file, 'w') as hdf_file:
            # Create a group to store the rem_tf_power_dict data
            rem_tf_power_dict_group = hdf_file.create_group('REM_tf_power_dict_group')

            # Save each key-value pair from rem_tf_power_dict into the HDF5 group
            for key, value in rem_tf_power_dict.items():
                # Use key as the dataset name and store the value
                rem_tf_power_dict_group[key] = value

      
    def process_subfolder_with_timing(self, subfolder):
        start_time = time.time()  # Start measuring time
        lfp_data, states_data = self.load_data(subfolder)
        rem_dict, rem_tf_power_dict = self.process_data(lfp_data, states_data)  # Process and get the rem_dict
        self.save_data(subfolder, rem_dict, rem_tf_power_dict)  # Save the rem_dict
        end_time = time.time()  # Stop measuring time
        elapsed_time = end_time - start_time  # Calculate elapsed time
        print(elapsed_time)  # Print the elapsed time in seconds
        # Return the dictionaries
        return rem_dict, rem_tf_power_dict



# Specify the input data directory and output directory.
data_dir = "E:/Donders/Data"  # Replace with your actual data directory path.
output_dir = "E:/Donders/Output"  # Replace with your desired output directory path.

# Create a DataProcessor object.
processor = DataProcessor(data_dir, output_dir)

# Process all data in the specified directories using parallel processing.
with concurrent.futures.ProcessPoolExecutor() as executor:
    subfolders = [subfolder for subfolder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, subfolder))]
    processed_data = []  # Create a list to collect processed data
    for subfolder in subfolders:
        result = processor.process_subfolder_with_timing(subfolder)  # Call the method and collect the dictionaries
        processed_data.append(result)  # Append the dictionaries to the list

# Now, 'processed_data' contains the dictionaries for each subfolder.

(1, 10802)
Key: REM 1, Value Type: <class 'dict'>


ValueError: Invalid data type in rem_dict for key 'REM 1': <class 'dict'>

NameError: name 'get_rem_states' is not defined

NameError: name 'elapsed_time' is not defined